# Evaluate *ritme* trials of all usecases


## Setup

In [ ]:
import warnings
import pandas as pd

from src.evaluate_trials import (
    plot_complexity_vs_metric,
    plot_trend_over_time,
    boxplot_metric,
    multi_boxplot_metric,
)

warnings.filterwarnings("ignore", category=FutureWarning)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
######## USER INPUTS ########

# path to extracted MLflow logs - with script extract_all_logs.sh
log_folder_location = "merged_u1_u2_runs_work_no2trac.csv"

# which usecase to analyze: "u1", "u2", "u3" or "all"
usecase = "u1"

# which samplers to analyse: "tpe", "random" or "all"
sampler = "tpe"
# whether to only analyze one model type or all (="all")
model_type = "all"

# how many trials to consider for complexity vs. performance plot
top_x = 1000

# figure saving dpi
dpi = 400
######## END USER INPUTS #####

In [ ]:
# set title
if usecase == "u1":
    title = "Usecase 1"
    best_model_type = "xgb"
elif usecase == "u2":
    title = "Usecase 2"
    best_model_type = "linreg"
elif usecase == "u3":
    title = "Usecase 3"
    best_model_type = "xgb"
else:
    title = "All usecases"

## Extract trial information

In [ ]:
# extract all trial information
all_trials = pd.read_csv(log_folder_location)
# sort by asc metrics.rmse_val
all_trials = all_trials.sort_values(by="metrics.rmse_val", ascending=True)
print(f"Found {all_trials.shape[0]} trials")

In [ ]:
if usecase != "all":
    print(f"Analyzing trials for usecase: {usecase}")
    all_trials = all_trials[all_trials["tags.experiment_tag"].str.startswith(usecase)]

if sampler != "all":
    print(f"Analyzing trials for sampler: {sampler}")
    all_trials = all_trials[all_trials["tags.experiment_tag"].str.contains(sampler)]

if model_type != "all":
    print(f"Analyzing trials for model type: {model_type}")
    all_trials = all_trials[all_trials["params.model"] == model_type]

print(f"Selected {all_trials.shape[0]} trials")

## Insights on performance

In [ ]:
fig, axes = multi_boxplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_specs=[
        ("params.data_aggregation", "Data aggregation"),
        ("params.data_selection", "Data selection"),
        ("params.data_transform", "Data transform"),
        ("params.data_enrich", "Data enrichment"),
        ("params.model", "Model type"),
    ],
    order_by_median=True,
    showfliers=False,
    title=title,
)
fig.savefig(
    f"result_figures/boxplot_all_trials_{usecase}.pdf", bbox_inches="tight", dpi=dpi
)

## Model complexity vs. performance: top X trials

In [ ]:
top_x_trials = all_trials.head(top_x)
top_x_trials["params.model"].value_counts()
figc, _ = plot_complexity_vs_metric(
    top_x_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col="params.model",
    group_name="Model type",
    n=top_x,
    figsize=(7, 6),
    title=title,
)

figc.savefig(
    f"result_figures/complexity_all_trials_{usecase}.pdf", bbox_inches="tight", dpi=dpi
)

In [ ]:
## Training over time

# metric = "rmse_val"
# for model in all_trials["params.model"].unique():
#     print(f"Plotting trend for model: {model}")
#     model_trials = all_trials[all_trials["params.model"] == model]
#     plot_trend_over_time(
#         model_trials, f"metrics.{metric}", window=100, title_prefix=f"Model: {model}", figsize=(12, 6
#     ))

# Top 1 trial

based on held-out test set performance

In [ ]:
top_1_trial = all_trials.loc[all_trials["params.model"] == best_model_type, :].head(1)

top_1_trial["metrics.nb_features"]

In [ ]:
top_1_trial["tags.experiment_tag"]